### Setup

In [ ]:
# Install basic dependencies.
%pip install gym-super-mario-bros nes-py matplotlib opencv-python
# Install dependencies for RL.
%pip install stable-baselines3
# Install PyTorch (customized).
%pip install torch torchvision torchaudio tensorboard

In [ ]:
# Import the game.
import gym_super_mario_bros
# Import the Joypad wrapper.
from nes_py.wrappers import JoypadSpace
# Import the simplified controls.
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Setup game.
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

### Preprocessing environment

In [ ]:
# Import GrayScaling Wrapper.
from gym.wrappers import GrayScaleObservation
# Import Vectorization wrappers.
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking.
from matplotlib import pyplot as plt

In [ ]:
# Create base environment.
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify the controls.
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale environment.
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside dummy environment.
env = DummyVecEnv([lambda: env])
# Stack frames.
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
# Use Matplotlib to show a game frame.
state = env.reset()
state.shape
plt.imshow(state[0])

In [ ]:
# Fill out frame stack.
for _ in range(20):
    state, reward, done, info = env.step([5])

# Use Matplotlib to show a visualization of the frame stack.
plt.figure(figsize=(20, 16))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx+1)
    plt.imshow(state[0][:, :, idx])

plt.show()

# Reset environment.
state = env.reset()

### Train RL model

In [ ]:
# Import os for file path management.
import os
# Import PPO for algos.
from stable_baselines3 import PPO
# Import BaseCallback for saving models.
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:


class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback.
callback = TrainAndLoggingCallback(check_freq=10_000, save_path=CHECKPOINT_DIR)

In [ ]:
# Start AI model.
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Train the AI model, the AI model starts to learn.
model.learn(total_timesteps=1_000_000, callback=callback)

### Test

In [ ]:
# Load last model.
model = PPO.load('./train/best_model_10000')

In [ ]:
# Fresh start.
state = env.reset()

In [ ]:
# Start the game.
state = env.reset()

# Loop through the game.
while True:

    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()